# Detecção de objetos em vídeos

## Ultralytics

In [ ]:
from ultralytics import YOLO

# Caminho do vídeo
video_path = "futebol.mp4"

# Carregar modelo pré-treinado YOLOv8
model = YOLO("yolov8s.pt")

# Rodar detecção no vídeo
results = model.predict(source=video_path, show=True, save=True, imgsz=1280, conf=0.25)



print("Processamento concluído! Resultados salvos na pasta 'runs/detect'.")


KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
import cv2

# Caminho do vídeo
video_path = "estac.mp4"

# Carregar modelo YOLOv8
model = YOLO("yolov8s.pt")

# Inicializar rastreamento
results = model.track(source=video_path, stream=True, tracker="bytetrack.yaml", conf=0.25)

# Dicionário para armazenar última posição de cada ID
last_positions = {}

# Abrir janela para exibir vídeo
for frame in results:
    img = frame.orig_img.copy()
    for box in frame.boxes:
        obj_id = int(box.id.cpu().numpy()[0]) if box.id is not None else None
        if obj_id is None:
            continue

        # Coordenadas atuais
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2  # centro do objeto

        # Verificar movimento
        moved = False
        if obj_id in last_positions:
            last_cx, last_cy = last_positions[obj_id]
            dist = ((cx - last_cx)**2 + (cy - last_cy)**2)**0.5
            if dist > 5:  # limiar de movimento (pixels)
                moved = True
        else:
            moved = True  # primeira vez, considera como movimento

        # Atualizar posição
        last_positions[obj_id] = (cx, cy)

        # Desenhar apenas se estiver em movimento
        if moved:
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(img, f"ID {obj_id}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Objetos em movimento", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()



video 1/1 (frame 1/3000) d:\YOLO\estac.mp4: 384x640 8 persons, 28 cars, 2 trucks, 1 traffic light, 83.5ms
video 1/1 (frame 2/3000) d:\YOLO\estac.mp4: 384x640 7 persons, 28 cars, 2 trucks, 61.7ms
video 1/1 (frame 3/3000) d:\YOLO\estac.mp4: 384x640 8 persons, 26 cars, 2 trucks, 57.8ms
video 1/1 (frame 4/3000) d:\YOLO\estac.mp4: 384x640 5 persons, 26 cars, 2 trucks, 60.1ms
video 1/1 (frame 5/3000) d:\YOLO\estac.mp4: 384x640 6 persons, 24 cars, 2 trucks, 55.8ms
video 1/1 (frame 6/3000) d:\YOLO\estac.mp4: 384x640 6 persons, 28 cars, 2 trucks, 58.4ms
video 1/1 (frame 7/3000) d:\YOLO\estac.mp4: 384x640 7 persons, 28 cars, 2 trucks, 1 traffic light, 55.9ms
video 1/1 (frame 8/3000) d:\YOLO\estac.mp4: 384x640 8 persons, 25 cars, 2 trucks, 55.2ms
video 1/1 (frame 9/3000) d:\YOLO\estac.mp4: 384x640 8 persons, 28 cars, 2 trucks, 60.6ms
video 1/1 (frame 10/3000) d:\YOLO\estac.mp4: 384x640 9 persons, 30 cars, 2 trucks, 56.2ms
video 1/1 (frame 11/3000) d:\YOLO\estac.mp4: 384x640 9 persons, 30 cars, 2

In [ ]:
import os
from collections import defaultdict, deque
import numpy as np
import cv2

from ultralytics import YOLO

# =========================
# CONFIGURAÇÕES PRINCIPAIS
# =========================
video_path = "estac.mp4"           # caminho do vídeo de entrada
model_path = "yolov8s.pt"          # modelo YOLOv8
tracker_cfg = "bytetrack.yaml"     # algoritmo de rastreamento
imgsz = 1280
conf = 0.25
device = "cpu"                         # 0 para GPU; use "cpu" se não tiver GPU

# Visual
TRAIL_LEN = 40                     # comprimento do rastro (nº de pontos na trajetória)
LINE_THICKNESS = 3                 # espessura da linha da trajetória
BOX_THICKNESS = 2                  # espessura da caixa
SHOW_WINDOW = True                 # mostrar janela com vídeo (requer tela)
SAVE_VIDEO = True                  # salvar vídeo com as trajetórias
OUTPUT_DIR = "runs/track_trails"   # pasta de saída
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Opcional: filtrar por classes do COCO (ex.: só 'person'=0, 'car'=2)
# Deixe como None para rastrear todas
FILTER_CLASSES = None  # exemplo: [0, 2]

# =========================
# FUNÇÕES AUXILIARES
# =========================
def color_by_id(track_id: int):
    """Gera uma cor BGR estável por ID."""
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return (int(c[0]), int(c[1]), int(c[2]))

def draw_trajectory(frame, points, color, thickness=2):
    """Desenha a polilinha da trajetória."""
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], isClosed=False, color=color, thickness=thickness)

def put_label(frame, text, x1, y1, color):
    """Desenha rótulo acima da caixa."""
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (tw, th), _ = cv2.getTextSize(text, font, scale, thickness)
    # fundo
    cv2.rectangle(frame, (x1, y1 - th - 6), (x1 + tw + 6, y1), color, -1)
    # texto
    cv2.putText(frame, text, (x1 + 3, y1 - 4), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

# =========================
# CARREGAR MODELO
# =========================
model = YOLO(model_path)

# =========================
# PREPARAR SAÍDA DE VÍDEO
# =========================
writer = None
frame_size = None
fps = 30.0

# Tentativa de obter FPS e tamanho do vídeo original
cap = cv2.VideoCapture(video_path)
if cap.isOpened():
    fps = cap.get(cv2.CAP_PROP_FPS) or fps
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (w, h)
cap.release()

if SAVE_VIDEO and frame_size is None:
    # Será definido no primeiro frame processado
    pass

# =========================
# ESTRUTURA PARA TRAJETÓRIAS
# =========================
# dicionário {track_id: deque([(x,y), ...], maxlen=TRAIL_LEN)}
trajectories = defaultdict(lambda: deque(maxlen=TRAIL_LEN))

# =========================
# LOOP DE RASTREAMENTO
# =========================
print("Iniciando rastreamento com trajetórias...")
frame_idx = 0

# stream=True -> gera frames iterativamente
# persist=True -> mantém IDs entre os frames no gerador
for result in model.track(
    source=video_path,
    imgsz=imgsz,
    conf=conf,
    device=device,
    tracker=tracker_cfg,
    stream=True,
    persist=True,
    classes=FILTER_CLASSES
):
    # frame original (BGR)
    frame = result.orig_img

    if frame_size is None:
        frame_size = (frame.shape[1], frame.shape[0])

    # Preparar writer quando possível
    if SAVE_VIDEO and writer is None and frame_size is not None:
        base = os.path.splitext(os.path.basename(video_path))[0]
        out_path = os.path.join(OUTPUT_DIR, f"{base}_traj.mp4")
        # Codec: mp4v (geralmente compatível); troque para 'XVID' se preferir AVI
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(out_path, fourcc, fps, frame_size)
        print(f"Salvando vídeo em: {out_path}")

    # Result boxes + IDs
    boxes = result.boxes
    if boxes is not None and boxes.id is not None:
        ids = boxes.id.int().cpu().tolist()
        xyxy = boxes.xyxy.cpu().numpy().astype(int)
        clss = boxes.cls.int().cpu().tolist()
        confs = boxes.conf.float().cpu().tolist()

        # nomes de classe
        names = result.names if hasattr(result, "names") else model.names

        for i, track_id in enumerate(ids):
            x1, y1, x2, y2 = xyxy[i]
            cls_id = clss[i]
            conf_i = confs[i] if i < len(confs) else None
            label = names.get(cls_id, str(cls_id)) if isinstance(names, dict) else (names[cls_id] if isinstance(names, list) and cls_id < len(names) else str(cls_id))
            color = color_by_id(track_id)

            # Centro do bbox
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Atualiza trajetória
            trajectories[track_id].append((cx, cy))

            # Desenhar bbox
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
            # Rótulo com classe, ID e confiança
            if conf_i is not None:
                put_label(frame, f"{label} #{track_id} {conf_i:.2f}", x1, y1, color)
            else:
                put_label(frame, f"{label} #{track_id}", x1, y1, color)

            # Desenhar trajetória
            draw_trajectory(frame, trajectories[track_id], color, thickness=LINE_THICKNESS)

    # Mostrar e/ou salvar
    if SHOW_WINDOW:
        cv2.imshow("YOLOv8 + ByteTrack (trajetórias)", frame)
        # Pressione 'q' para sair
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if SAVE_VIDEO and writer is not None:
        # redimensiona caso necessário
        if (frame.shape[1], frame.shape[0]) != frame_size:
            frame = cv2.resize(frame, frame_size)
        writer.write(frame)

    frame_idx += 1

# Finalização
if writer is not None:
    writer.release()
cv2.destroyAllWindows()

print("Processamento concluído! Vídeo com trajetórias salvo na pasta:", OUTPUT_DIR)


Iniciando rastreamento com trajetórias...

video 1/1 (frame 1/3000) d:\YOLO\estac.mp4: 736x1280 9 persons, 40 cars, 3 trucks, 1 traffic light, 1 backpack, 1 chair, 211.0ms
Salvando vídeo em: runs/track_trails\estac_traj.mp4
video 1/1 (frame 2/3000) d:\YOLO\estac.mp4: 736x1280 8 persons, 37 cars, 3 trucks, 1 traffic light, 1 backpack, 1 chair, 206.1ms
video 1/1 (frame 3/3000) d:\YOLO\estac.mp4: 736x1280 7 persons, 36 cars, 3 trucks, 1 traffic light, 1 backpack, 1 chair, 189.2ms
video 1/1 (frame 4/3000) d:\YOLO\estac.mp4: 736x1280 7 persons, 37 cars, 3 trucks, 1 traffic light, 1 backpack, 1 chair, 186.2ms
video 1/1 (frame 5/3000) d:\YOLO\estac.mp4: 736x1280 7 persons, 39 cars, 2 trucks, 1 traffic light, 1 backpack, 1 chair, 202.0ms
video 1/1 (frame 6/3000) d:\YOLO\estac.mp4: 736x1280 8 persons, 40 cars, 2 trucks, 1 traffic light, 1 backpack, 1 chair, 199.6ms
video 1/1 (frame 7/3000) d:\YOLO\estac.mp4: 736x1280 8 persons, 38 cars, 3 trucks, 1 traffic light, 1 backpack, 1 chair, 206.3ms
vi

KeyboardInterrupt: 

In [ ]:
import os
from collections import defaultdict, deque
import csv
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# =========================
# CONFIGURAÇÕES PRINCIPAIS
# =========================
video_path = "estac.mp4"           # caminho do vídeo de entrada
model_path = "yolov8n.pt"          # use 'yolov8n.pt' (mais leve na CPU) ou 'yolov8s.pt'
tracker_cfg = "bytetrack.yaml"     # algoritmo de rastreamento (depende de 'lap')
imgsz = 960                        # reduza p/ 640 se precisar de mais desempenho
conf = 0.30                        # confiança mínima (aumentar reduz falsos positivos)
device = "cpu"                     # você pediu CPU; se quiser automático: 0 if torch.cuda.is_available() else "cpu"

# Visual
TRAIL_LEN = 40                     # comprimento do rastro (nº de pontos)
LINE_THICKNESS = 3                 # espessura da linha da trajetória
BOX_THICKNESS = 2                  # espessura da caixa
SHOW_WINDOW = True                 # exibir janela (defina False em servidor/Colab)
SAVE_VIDEO = True                  # salvar vídeo com trajetórias
OUTPUT_DIR = "runs/track_trails"   # pasta de saída
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filtro de classes do COCO (None = todas). Ex.: [0] só pessoas, [2, 7] carros/caminhões
FILTER_CLASSES = None

# =========================
# PARÂMETROS DE MOVIMENTO
# =========================
# Limiar de deslocamento entre frames (em pixels) para considerar "em movimento".
# Se a câmera tremer muito, aumente (ex.: 4~8 px).
MOTION_PIX_THRESH = 3.0

# Persistência opcional: se quiser manter a caixa visível por alguns frames após parar,
# defina LINGER_FRAMES > 0. Com 0, some imediatamente quando parar.
LINGER_FRAMES = 0

# =========================
# EXPORTAÇÃO CSV
# =========================
EXPORT_CSV = True
CSV_BASENAME = None  # None -> auto pelo nome do vídeo

# =========================
# FUNÇÕES AUXILIARES
# =========================
def color_by_id(track_id: int):
    """Gera uma cor BGR estável por ID."""
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return (int(c[0]), int(c[1]), int(c[2]))

def draw_trajectory(frame, points, color, thickness=2):
    """Desenha a polilinha da trajetória."""
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], isClosed=False, color=color, thickness=thickness)

def put_label(frame, text, x1, y1, color):
    """Desenha rótulo acima da caixa."""
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (tw, th), _ = cv2.getTextSize(text, font, scale, thickness)
    # fundo do rótulo
    y1_txt = max(th + 8, y1)  # evita coordenadas negativas
    cv2.rectangle(frame, (x1, y1_txt - th - 6), (x1 + tw + 6, y1_txt), color, -1)
    # texto
    cv2.putText(frame, text, (x1 + 3, y1_txt - 4), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

# =========================
# CARREGAR MODELO
# =========================
print("Usando dispositivo:", device if device == "cpu" else "GPU (cuda)")
model = YOLO(model_path)

# =========================
# PREPARAR SAÍDA DE VÍDEO E CSV
# =========================
writer = None
frame_size = None
fps = 30.0

cap_tmp = cv2.VideoCapture(video_path)
if cap_tmp.isOpened():
    fps = cap_tmp.get(cv2.CAP_PROP_FPS) or fps
    w = int(cap_tmp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap_tmp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (w, h)
cap_tmp.release()

base = os.path.splitext(os.path.basename(video_path))[0]
if SAVE_VIDEO:
    out_path = os.path.join(OUTPUT_DIR, f"{base}_traj_moveonly.mp4")
if EXPORT_CSV:
    csv_name = CSV_BASENAME or f"{base}_moveonly.csv"
    csv_path = os.path.join(OUTPUT_DIR, csv_name)
    csv_file = open(csv_path, "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame", "id", "x1", "y1", "x2", "y2", "cx", "cy", "class", "conf", "dist_px", "moved"])

# =========================
# ESTRUTURAS DE RASTRO E MOVIMENTO
# =========================
trajectories = defaultdict(lambda: deque(maxlen=TRAIL_LEN))  # {id: deque[(cx,cy), ...]}
last_positions = {}          # {id: (cx, cy)}
linger_left = defaultdict(int)  # {id: frames restantes de persistência}

# =========================
# LOOP DE RASTREAMENTO
# =========================
print("Iniciando rastreamento (mostrando apenas objetos em movimento)...")

try:
    for result in model.track(
        source=video_path,
        imgsz=imgsz,
        conf=conf,
        device=device,
        tracker=tracker_cfg,
        stream=True,
        persist=True,
        classes=FILTER_CLASSES
    ):
        frame = result.orig_img

        # Inicializa writer quando tiver tamanho
        if frame_size is None and frame is not None:
            frame_size = (frame.shape[1], frame.shape[0])
        if SAVE_VIDEO and writer is None and frame_size is not None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'XVID' para .avi, se preferir
            writer = cv2.VideoWriter(out_path, fourcc, fps, frame_size)
            print(f"Salvando vídeo em: {out_path}")

        boxes = result.boxes
        frame_idx = int(getattr(result, "frame_idx", 0))  # pode não existir; não crítico

        if boxes is None or boxes.id is None:
            # nada rastreado neste frame
            if SHOW_WINDOW:
                cv2.imshow("YOLOv8 + ByteTrack (trajetórias - apenas movimento)", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            if SAVE_VIDEO and writer is not None:
                if (frame.shape[1], frame.shape[0]) != frame_size:
                    frame = cv2.resize(frame, frame_size)
                writer.write(frame)
            continue

        ids = boxes.id.int().cpu().tolist()
        xyxy = boxes.xyxy.cpu().numpy().astype(int)
        clss = boxes.cls.int().cpu().tolist() if boxes.cls is not None else [None] * len(ids)
        confs = boxes.conf.float().cpu().tolist() if boxes.conf is not None else [None] * len(ids)

        names = result.names if hasattr(result, "names") else model.names

        # Processa cada trilha
        for i, track_id in enumerate(ids):
            x1, y1, x2, y2 = xyxy[i]
            cls_id = clss[i]
            conf_i = confs[i] if i < len(confs) else None
            label = (
                names.get(cls_id, str(cls_id)) if isinstance(names, dict)
                else (names[cls_id] if isinstance(names, list) and cls_id is not None and cls_id < len(names) else str(cls_id))
            )
            color = color_by_id(track_id)

            # centro do bbox
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # mede deslocamento desde o último frame visto para este ID
            dist = 0.0
            moved = False
            if track_id in last_positions:
                last_cx, last_cy = last_positions[track_id]
                dx = cx - last_cx
                dy = cy - last_cy
                dist = float((dx * dx + dy * dy) ** 0.5)
                moved = dist > MOTION_PIX_THRESH
            # atualiza última posição
            last_positions[track_id] = (cx, cy)

            if moved:
                # atualiza trajetória só quando houver movimento (rastro mostra o caminho)
                trajectories[track_id].append((cx, cy))
                linger_left[track_id] = LINGER_FRAMES  # reseta persistência
            else:
                # se quiser persistência visual por alguns frames após parar:
                if LINGER_FRAMES > 0 and linger_left[track_id] > 0:
                    linger_left[track_id] -= 1
                else:
                    # sem movimento e sem persistir -> não desenha nada e segue
                    continue

            # Desenha (somente se em movimento ou na persistência)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
            if conf_i is not None:
                put_label(frame, f"{label} #{track_id} {conf_i:.2f}", x1, y1, color)
            else:
                put_label(frame, f"{label} #{track_id}", x1, y1, color)
            draw_trajectory(frame, trajectories[track_id], color, thickness=LINE_THICKNESS)

            # CSV: salva apenas quando há movimento (ou durante a persistência, se LINGER_FRAMES > 0)
            if EXPORT_CSV:
                csv_writer.writerow([
                    frame_idx, track_id, x1, y1, x2, y2, cx, cy,
                    label, f"{conf_i:.4f}" if conf_i is not None else "",
                    f"{dist:.3f}", moved
                ])

        # Mostrar e/ou salvar frame
        if SHOW_WINDOW:
            cv2.imshow("YOLOv8 + ByteTrack (trajetórias - apenas movimento)", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if SAVE_VIDEO and writer is not None:
            if (frame.shape[1], frame.shape[0]) != frame_size:
                frame = cv2.resize(frame, frame_size)
            writer.write(frame)

finally:
    if writer is not None:
        writer.release()
    if EXPORT_CSV:
        csv_file.close()
        print(f"CSV salvo em: {csv_path}")
    cv2.destroyAllWindows()
    print("Processamento concluído! Saídas na pasta:", OUTPUT_DIR)


Usando dispositivo: cpu
Iniciando rastreamento (mostrando apenas objetos em movimento)...

video 1/1 (frame 1/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 26 cars, 1 truck, 90.9ms
Salvando vídeo em: runs/track_trails\estac_traj_moveonly.mp4
video 1/1 (frame 2/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 22 cars, 1 truck, 56.0ms
video 1/1 (frame 3/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 23 cars, 1 truck, 57.9ms
video 1/1 (frame 4/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 57.4ms
video 1/1 (frame 5/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 20 cars, 1 truck, 56.5ms
video 1/1 (frame 6/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 21 cars, 1 truck, 55.2ms
video 1/1 (frame 7/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 55.7ms
video 1/1 (frame 8/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 24 cars, 1 truck, 59.6ms
video 1/1 (frame 9/3000) d:\YOLO\estac.mp4: 544x960 6 persons, 17 cars, 1 truck, 59.1ms
video 1/1 (frame 10/3000) d:\YOLO\estac.mp4: 544x960 8 p

In [ ]:
# salva a trajetoria 


import os
from collections import defaultdict, deque
import csv
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# =========================
# CONFIGURAÇÕES PRINCIPAIS
# =========================
video_path = "estac.mp4"           # caminho do vídeo
model_path = "yolov8n.pt"          # use 'yolov8n.pt' para CPU
tracker_cfg = "bytetrack.yaml"
imgsz = 960
conf = 0.30
device = "cpu"                     # CPU forçado

# Visual
TRAIL_LEN = 40
LINE_THICKNESS = 3
BOX_THICKNESS = 2
SHOW_WINDOW = True
SAVE_VIDEO = True
OUTPUT_DIR = "runs/track_trails"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Movimento
MOTION_PIX_THRESH = 3.0
LINGER_FRAMES = 0

# Exportação
EXPORT_CSV = True
ACCUMULATED_VIDEO = True
CSV_BASENAME = None

# =========================
# FUNÇÕES AUXILIARES
# =========================
def color_by_id(track_id: int):
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return (int(c[0]), int(c[1]), int(c[2]))

def draw_trajectory(frame, points, color, thickness=2):
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], isClosed=False, color=color, thickness=thickness)

def put_label(frame, text, x1, y1, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (tw, th), _ = cv2.getTextSize(text, font, scale, thickness)
    y1_txt = max(th + 8, y1)
    cv2.rectangle(frame, (x1, y1_txt - th - 6), (x1 + tw + 6, y1_txt), color, -1)
    cv2.putText(frame, text, (x1 + 3, y1_txt - 4), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

# =========================
# CARREGAR MODELO
# =========================
print("Usando dispositivo:", device)
model = YOLO(model_path)

# =========================
# PREPARAR SAÍDAS
# =========================
writer = None
writer_accum = None
frame_size = None
fps = 30.0

cap_tmp = cv2.VideoCapture(video_path)
if cap_tmp.isOpened():
    fps = cap_tmp.get(cv2.CAP_PROP_FPS) or fps
    w = int(cap_tmp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap_tmp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (w, h)
cap_tmp.release()

base = os.path.splitext(os.path.basename(video_path))[0]
if SAVE_VIDEO:
    out_path = os.path.join(OUTPUT_DIR, f"{base}_traj_moveonly.mp4")
if ACCUMULATED_VIDEO:
    out_accum_path = os.path.join(OUTPUT_DIR, f"{base}_traj_accum.mp4")
if EXPORT_CSV:
    csv_name = CSV_BASENAME or f"{base}_moveonly.csv"
    csv_path = os.path.join(OUTPUT_DIR, csv_name)
    csv_file = open(csv_path, "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame", "id", "x1", "y1", "x2", "y2", "cx", "cy", "class", "conf", "dist_px", "moved"])

# =========================
# ESTRUTURAS
# =========================
trajectories = defaultdict(lambda: deque(maxlen=TRAIL_LEN))
last_positions = {}
linger_left = defaultdict(int)

# Para imagem final e vídeo acumulado
accum_canvas = np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8)

# =========================
# LOOP DE RASTREAMENTO
# =========================
print("Iniciando rastreamento (somente objetos em movimento)...")

try:
    for result in model.track(
        source=video_path,
        imgsz=imgsz,
        conf=conf,
        device=device,
        tracker=tracker_cfg,
        stream=True,
        persist=True
    ):
        frame = result.orig_img

        if frame_size is None and frame is not None:
            frame_size = (frame.shape[1], frame.shape[0])
        if SAVE_VIDEO and writer is None and frame_size is not None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            writer = cv2.VideoWriter(out_path, fourcc, fps, frame_size)
            if ACCUMULATED_VIDEO:
                writer_accum = cv2.VideoWriter(out_accum_path, fourcc, fps, frame_size)
            print(f"Salvando vídeo em: {out_path}")

        boxes = result.boxes
        frame_idx = int(getattr(result, "frame_idx", 0))

        if boxes is None or boxes.id is None:
            if SHOW_WINDOW:
                cv2.imshow("YOLOv8 + ByteTrack (trajetórias)", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            if SAVE_VIDEO and writer is not None:
                writer.write(frame)
                if ACCUMULATED_VIDEO:
                    writer_accum.write(accum_canvas)
            continue

        ids = boxes.id.int().cpu().tolist()
        xyxy = boxes.xyxy.cpu().numpy().astype(int)
        clss = boxes.cls.int().cpu().tolist() if boxes.cls is not None else [None] * len(ids)
        confs = boxes.conf.float().cpu().tolist() if boxes.conf is not None else [None] * len(ids)

        names = result.names if hasattr(result, "names") else model.names

        for i, track_id in enumerate(ids):
            x1, y1, x2, y2 = xyxy[i]
            cls_id = clss[i]
            conf_i = confs[i] if i < len(confs) else None
            label = (
                names.get(cls_id, str(cls_id)) if isinstance(names, dict)
                else (names[cls_id] if isinstance(names, list) and cls_id is not None and cls_id < len(names) else str(cls_id))
            )
            color = color_by_id(track_id)

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            dist = 0.0
            moved = False
            if track_id in last_positions:
                last_cx, last_cy = last_positions[track_id]
                dx = cx - last_cx
                dy = cy - last_cy
                dist = float((dx * dx + dy * dy) ** 0.5)
                moved = dist > MOTION_PIX_THRESH
            last_positions[track_id] = (cx, cy)

            if moved:
                trajectories[track_id].append((cx, cy))
                linger_left[track_id] = LINGER_FRAMES
            else:
                if LINGER_FRAMES > 0 and linger_left[track_id] > 0:
                    linger_left[track_id] -= 1
                else:
                    continue

            # Desenha no frame atual
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
            put_label(frame, f"{label} #{track_id} {conf_i:.2f}" if conf_i else f"{label} #{track_id}", x1, y1, color)
            draw_trajectory(frame, trajectories[track_id], color, thickness=LINE_THICKNESS)

            # Atualiza canvas acumulado
            draw_trajectory(accum_canvas, trajectories[track_id], color, thickness=2)

            if EXPORT_CSV:
                csv_writer.writerow([frame_idx, track_id, x1, y1, x2, y2, cx, cy, label,
                                      f"{conf_i:.4f}" if conf_i else "", f"{dist:.3f}", moved])

        if SHOW_WINDOW:
            cv2.imshow("YOLOv8 + ByteTrack (trajetórias)", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if SAVE_VIDEO and writer is not None:
            writer.write(frame)
            if ACCUMULATED_VIDEO:
                writer_accum.write(accum_canvas)

finally:
    if writer is not None:
        writer.release()
    if writer_accum is not None:
        writer_accum.release()
    if EXPORT_CSV:
        csv_file.close()
        print(f"CSV salvo em: {csv_path}")

    # Salva imagem final com todas as trajetórias
    traj_path = os.path.join(OUTPUT_DIR, f"{base}_trajetorias.png")
    cv2.imwrite(traj_path, accum_canvas)
    print(f"Imagem com trajetórias salva em: {traj_path}")

    cv2.destroyAllWindows()
    print("Processamento concluído! Saídas na pasta:", OUTPUT_DIR)


Usando dispositivo: cpu
Iniciando rastreamento (somente objetos em movimento)...

video 1/1 (frame 1/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 26 cars, 1 truck, 62.1ms
Salvando vídeo em: runs/track_trails\estac_traj_moveonly.mp4
video 1/1 (frame 2/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 22 cars, 1 truck, 55.8ms
video 1/1 (frame 3/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 23 cars, 1 truck, 56.9ms
video 1/1 (frame 4/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 57.6ms
video 1/1 (frame 5/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 20 cars, 1 truck, 57.3ms
video 1/1 (frame 6/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 21 cars, 1 truck, 55.4ms
video 1/1 (frame 7/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 51.5ms
video 1/1 (frame 8/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 24 cars, 1 truck, 52.9ms
video 1/1 (frame 9/3000) d:\YOLO\estac.mp4: 544x960 6 persons, 17 cars, 1 truck, 56.4ms
video 1/1 (frame 10/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 2

In [ ]:
# melhoria de parametros
import os
from collections import defaultdict, deque
import csv
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# =========================
# CONFIGURAÇÕES PRINCIPAIS
# =========================
video_path = "estac.mp4"
model_path = "yolov8n.pt"
tracker_cfg = "bytetrack.yaml"
imgsz = 960
conf = 0.30
device = "cpu"

# Visual
TRAIL_LEN = 40
LINE_THICKNESS = 3
BOX_THICKNESS = 2
SHOW_WINDOW = True
SAVE_VIDEO = True
OUTPUT_DIR = "runs/track_advanced"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Movimento
MOTION_PIX_THRESH = 3.0
LINGER_FRAMES = 0

# Exportação
EXPORT_CSV = True
CSV_BASENAME = None

# Escala opcional para converter px → metros (ex.: 0.02 m/px)
PIXEL_TO_METER = None  # defina um valor se souber a escala

# =========================
# FUNÇÕES AUXILIARES
# =========================
def color_by_id(track_id: int):
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return (int(c[0]), int(c[1]), int(c[2]))

def draw_trajectory(frame, points, color, thickness=2):
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], isClosed=False, color=color, thickness=thickness)

def put_label(frame, text, x1, y1, color):
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (tw, th), _ = cv2.getTextSize(text, font, scale, thickness)
    y1_txt = max(th + 8, y1)
    cv2.rectangle(frame, (x1, y1_txt - th - 6), (x1 + tw + 6, y1_txt), color, -1)
    cv2.putText(frame, text, (x1 + 3, y1_txt - 4), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

def smooth_point(points, window=3):
    """Suaviza pontos usando média móvel."""
    if len(points) < window:
        return points[-1]
    arr = np.array(points)[-window:]
    return tuple(arr.mean(axis=0).astype(int))

# =========================
# CARREGAR MODELO
# =========================
print("Usando dispositivo:", device)
model = YOLO(model_path)

# =========================
# PREPARAR SAÍDAS
# =========================
writer = None
frame_size = None
fps = 30.0

cap_tmp = cv2.VideoCapture(video_path)
if cap_tmp.isOpened():
    fps = cap_tmp.get(cv2.CAP_PROP_FPS) or fps
    w = int(cap_tmp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap_tmp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (w, h)
cap_tmp.release()

base = os.path.splitext(os.path.basename(video_path))[0]
if SAVE_VIDEO:
    out_path = os.path.join(OUTPUT_DIR, f"{base}_advanced.mp4")
if EXPORT_CSV:
    csv_name = CSV_BASENAME or f"{base}_advanced.csv"
    csv_path = os.path.join(OUTPUT_DIR, csv_name)
    csv_file = open(csv_path, "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame", "id", "cx", "cy", "speed_px_per_s", "speed_m_per_s", "distance_px", "distance_m", "class", "conf"])

# =========================
# ESTRUTURAS
# =========================
trajectories = defaultdict(lambda: deque(maxlen=TRAIL_LEN))
last_positions = {}
distances = defaultdict(float)  # distância acumulada por ID
last_frame_time = defaultdict(int)

# =========================
# LOOP DE RASTREAMENTO
# =========================
print("Iniciando rastreamento avançado...")

try:
    for result in model.track(
        source=video_path,
        imgsz=imgsz,
        conf=conf,
        device=device,
        tracker=tracker_cfg,
        stream=True,
        persist=True
    ):
        frame = result.orig_img
        frame_idx = int(getattr(result, "frame_idx", 0))

        if frame_size is None and frame is not None:
            frame_size = (frame.shape[1], frame.shape[0])
        if SAVE_VIDEO and writer is None and frame_size is not None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            writer = cv2.VideoWriter(out_path, fourcc, fps, frame_size)
            print(f"Salvando vídeo em: {out_path}")

        boxes = result.boxes
        if boxes is None or boxes.id is None:
            if SHOW_WINDOW:
                cv2.imshow("YOLOv8 Avançado", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            if SAVE_VIDEO and writer is not None:
                writer.write(frame)
            continue

        ids = boxes.id.int().cpu().tolist()
        xyxy = boxes.xyxy.cpu().numpy().astype(int)
        clss = boxes.cls.int().cpu().tolist() if boxes.cls is not None else [None] * len(ids)
        confs = boxes.conf.float().cpu().tolist() if boxes.conf is not None else [None] * len(ids)

        names = result.names if hasattr(result, "names") else model.names
        speeds = []

        for i, track_id in enumerate(ids):
            x1, y1, x2, y2 = xyxy[i]
            cls_id = clss[i]
            conf_i = confs[i] if i < len(confs) else None
            label = (
                names.get(cls_id, str(cls_id)) if isinstance(names, dict)
                else (names[cls_id] if isinstance(names, list) and cls_id is not None and cls_id < len(names) else str(cls_id))
            )
            color = color_by_id(track_id)

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Suavização
            trajectories[track_id].append((cx, cy))
            smooth_cx, smooth_cy = smooth_point(trajectories[track_id])

            # Cálculo de velocidade e distância
            speed_px = 0.0
            if track_id in last_positions:
                dx = smooth_cx - last_positions[track_id][0]
                dy = smooth_cy - last_positions[track_id][1]
                dist = (dx ** 2 + dy ** 2) ** 0.5
                distances[track_id] += dist
                speed_px = dist * fps  # px/s
            last_positions[track_id] = (smooth_cx, smooth_cy)

            speed_m = speed_px * PIXEL_TO_METER if PIXEL_TO_METER else None
            dist_m = distances[track_id] * PIXEL_TO_METER if PIXEL_TO_METER else None
            speeds.append(speed_px)

            # Desenha
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
            speed_text = f"{speed_px:.1f}px/s" if not PIXEL_TO_METER else f"{speed_m:.2f}m/s"
            put_label(frame, f"{label} #{track_id} {speed_text}", x1, y1, color)
            draw_trajectory(frame, trajectories[track_id], color, thickness=LINE_THICKNESS)

            if EXPORT_CSV:
                csv_writer.writerow([frame_idx, track_id, smooth_cx, smooth_cy, f"{speed_px:.2f}",
                                      f"{speed_m:.3f}" if speed_m else "", f"{distances[track_id]:.2f}",
                                      f"{dist_m:.3f}" if dist_m else "", label, f"{conf_i:.4f}" if conf_i else ""])

        # Overlay de estatísticas
        if speeds:
            avg_speed = sum(speeds) / len(speeds)
            cv2.putText(frame, f"Objetos ativos: {len(speeds)} | Velocidade média: {avg_speed:.1f}px/s",
                        (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        if SHOW_WINDOW:
            cv2.imshow("YOLOv8 Avançado", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if SAVE_VIDEO and writer is not None:
            writer.write(frame)

finally:
    if writer is not None:
        writer.release()
    if EXPORT_CSV:
        csv_file.close()
        print(f"CSV salvo em: {csv_path}")
    cv2.destroyAllWindows()
    print("Processamento concluído! Saídas na pasta:", OUTPUT_DIR)



Usando dispositivo: cpu
Iniciando rastreamento avançado...

video 1/1 (frame 1/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 26 cars, 1 truck, 64.1ms
Salvando vídeo em: runs/track_advanced\estac_advanced.mp4
video 1/1 (frame 2/3000) d:\YOLO\estac.mp4: 544x960 5 persons, 22 cars, 1 truck, 51.4ms
video 1/1 (frame 3/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 23 cars, 1 truck, 52.5ms
video 1/1 (frame 4/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 52.6ms
video 1/1 (frame 5/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 20 cars, 1 truck, 52.9ms
video 1/1 (frame 6/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 21 cars, 1 truck, 50.4ms
video 1/1 (frame 7/3000) d:\YOLO\estac.mp4: 544x960 7 persons, 20 cars, 1 truck, 49.1ms
video 1/1 (frame 8/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 24 cars, 1 truck, 52.3ms
video 1/1 (frame 9/3000) d:\YOLO\estac.mp4: 544x960 6 persons, 17 cars, 1 truck, 54.7ms
video 1/1 (frame 10/3000) d:\YOLO\estac.mp4: 544x960 8 persons, 21 cars, 1 truck, 50.8ms
v

In [5]:
import cv2

video_path = "mask_planta.png"
cap = cv2.VideoCapture(video_path)

ret, frame = cap.read()
cap.release()

# Redimensionar a imagem para caber melhor na tela
scale_percent = 70  # ajuste esse valor conforme necessário
width = int(frame.shape[1] * scale_percent / 100)
height = int(frame.shape[0] * scale_percent / 100)
dim = (width, height)
resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

coords = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Ajustar coordenadas para o tamanho original
        original_x = int(x * frame.shape[1] / resized_frame.shape[1])
        original_y = int(y * frame.shape[0] / resized_frame.shape[0])
        coords.append((original_x, original_y))
        print(f"Ponto: {original_x}, {original_y}")

cv2.imshow("Selecione os pontos", resized_frame)
cv2.setMouseCallback("Selecione os pontos", click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Coordenadas selecionadas:", coords)


Ponto: 27, 770
Ponto: 284, 771
Ponto: 522, 772
Ponto: 525, 541
Ponto: 292, 540
Ponto: 31, 537
Ponto: 32, 305
Ponto: 284, 298
Ponto: 524, 302
Ponto: 34, 30
Ponto: 282, 31
Coordenadas selecionadas: [(27, 770), (284, 771), (522, 772), (525, 541), (292, 540), (31, 537), (32, 305), (284, 298), (524, 302), (34, 30), (282, 31)]


In [ ]:
# # CONSTRUÇÃO

# import os
# from collections import defaultdict
# import numpy as np
# import cv2
# from ultralytics import YOLO

# # =========================
# # CONFIGURAÇÕES
# # =========================
# video_path = "constru.mp4"
# model_path = "yolov8l.pt"
# tracker_cfg = "bytetrack.yaml"
# imgsz = 1280
# conf = 0.30
# device = "cpu"

# # Visual
# LINE_THICKNESS = 3
# BOX_THICKNESS = 2
# SHOW_WINDOW = True
# SAVE_VIDEO = True
# OUTPUT_DIR = "runs/track_advanced"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # =========================
# # PONTOS PARA HOMOGRAFIA
# # =========================
# pts_video = np.array([
#     (2053, 1738),
#     (2703, 1868),
#     (3650, 1574),
#     (2787, 1372),
#     (3604, 1218),
#     (1768, 714),
#     (808, 1633),
#     (168, 1812),
#     (878, 2142)
# ], dtype=np.float32)

# pts_planta = np.array([
#     (937, 1153),
#     (1241, 1156),
#     (1244, 615),
#     (932, 610),
#     (945, 149),
#     (146, 154),
#     (612, 1153),
#     (610, 1611),
#     (932, 1611)
# ], dtype=np.float32)

# H, _ = cv2.findHomography(pts_video, pts_planta)

# # Carregar imagem da planta
# planta_img = cv2.imread("planta_constru.jpeg")  # altere para seu arquivo real
# planta_acumulada = planta_img.copy()

# # =========================
# # FUNÇÕES
# # =========================
# def color_by_id(track_id: int):
#     np.random.seed(int(track_id) * 1117)
#     c = np.random.randint(0, 255, 3).tolist()
#     return tuple(map(int, c))

# def draw_trajectory(frame, points, color, thickness=2):
#     if len(points) > 1:
#         pts = np.array(points, dtype=np.int32)
#         cv2.polylines(frame, [pts], False, color, thickness)

# def smooth_point(points, window=3):
#     if len(points) < window:
#         return points[-1]
#     arr = np.array(points)[-window:]
#     return tuple(arr.mean(axis=0).astype(int))

# # =========================
# # CARREGAR MODELO
# # =========================
# model = YOLO(model_path)

# # Estruturas
# trajectories_video = defaultdict(list)  # acumula todos os pontos no vídeo
# trajectories_planta = defaultdict(list)  # acumula todos os pontos na planta

# # =========================
# # LOOP DE RASTREAMENTO
# # =========================
# cap = cv2.VideoCapture(video_path)
# fps = cap.get(cv2.CAP_PROP_FPS)
# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer = None
# if SAVE_VIDEO:
#     out_path = os.path.join(OUTPUT_DIR, "video_combinado.mp4")
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     writer = cv2.VideoWriter(out_path, fourcc, fps, (w + planta_img.shape[1], h))  # altura do vídeo garantida

# for result in model.track(
#     source=video_path,
#     imgsz=imgsz,
#     conf=conf,
#     device=device,
#     tracker=tracker_cfg,
#     stream=True,
#     persist=True
# ):
#     frame = result.orig_img.copy()

#     boxes = result.boxes
#     if boxes is None or boxes.id is None:
#         continue

#     ids = boxes.id.int().cpu().tolist()
#     xyxy = boxes.xyxy.cpu().numpy().astype(int)

#     for i, track_id in enumerate(ids):
#         x1, y1, x2, y2 = xyxy[i]
#         cx = int((x1 + x2) / 2)
#         cy = int((y1 + y2) / 2)

#         # Salvar no vídeo
#         trajectories_video[track_id].append((cx, cy))
#         smooth_cx, smooth_cy = smooth_point(trajectories_video[track_id])

#         # Converter para coordenada da planta e acumular
#         ponto_vid = np.array([[[smooth_cx, smooth_cy]]], dtype=np.float32)
#         ponto_pla = cv2.perspectiveTransform(ponto_vid, H)[0][0]
#         px_planta, py_planta = int(ponto_pla[0]), int(ponto_pla[1])
#         trajectories_planta[track_id].append((px_planta, py_planta))

#         # Desenha no vídeo
#         color = color_by_id(track_id)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
#         draw_trajectory(frame, trajectories_video[track_id], color, LINE_THICKNESS)

#         # Desenha na planta acumulada
#         draw_trajectory(planta_acumulada, trajectories_planta[track_id], color, LINE_THICKNESS)
#         cv2.circle(planta_acumulada, (px_planta, py_planta), 4, color, -1)

#     # Redimensionar planta para mesma altura do vídeo
#     planta_resized = cv2.resize(planta_acumulada, (planta_acumulada.shape[1], h))
#     combined = np.zeros((h, w + planta_resized.shape[1], 3), dtype=np.uint8)
#     combined[:h, :w] = frame
#     combined[:h, w:w+planta_resized.shape[1]] = planta_resized

#     if SHOW_WINDOW:
#         cv2.imshow("Video + Planta", combined)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     if SAVE_VIDEO:
#         writer.write(combined)

# cap.release()
# if writer:
#     writer.release()
# cv2.destroyAllWindows()

# # =========================
# # SALVAR PLANTA COM RASTROS
# # =========================
# planta_saida = os.path.join(OUTPUT_DIR, "planta_com_rastros.png")
# cv2.imwrite(planta_saida, planta_acumulada)
# print(f"Planta com rastros salva em: {planta_saida}")
# print("Processo concluído! O vídeo combinado foi salvo e a planta acumulada também.")

In [1]:
import os
from collections import defaultdict
import numpy as np
import cv2
from ultralytics import YOLO

# =========================
# CONFIGURAÇÕES REFINADAS
# =========================
video_path = "video cortado.mp4"
model_path = "yolov8l.pt"  # modelo mais robusto
tracker_cfg = "bytetrack.yaml"
imgsz = 1280  # resolução maior
conf = 0.40   # confiança mais alta
device = "cpu"  # altere para "cuda" se tiver GPU

# Visual
LINE_THICKNESS = 3
BOX_THICKNESS = 2
SHOW_WINDOW = True
SAVE_VIDEO = True
OUTPUT_DIR = "runs/refino4"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# PONTOS PARA HOMOGRAFIA
# =========================
pts_video = np.array([(16, 2886), (3223, 2630), (3820, 2253), (3223, 1376), (2360, 1360), (743, 1330), (1173, 900), (2010, 956), (2760, 990), (1313, 790), (1816, 790)], dtype=np.float32)

pts_planta = np.array([(27, 770), (284, 771), (522, 772), (525, 541), (292, 540), (31, 537), (32, 305), (284, 298), (524, 302), (34, 30), (282, 31)], dtype=np.float32)

H, _ = cv2.findHomography(pts_video, pts_planta)


# Carregar imagem da planta
planta_img = cv2.imread("mask_planta.png")
planta_acumulada = planta_img.copy()

# =========================
# FUNÇÕES REFINADAS
# =========================
def color_by_id(track_id: int):
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return tuple(map(int, c))

def draw_trajectory(frame, points, color, thickness=2):
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], False, color, thickness)

def smooth_point_weighted(points, window=5):
    if len(points) < window:
        return points[-1]
    arr = np.array(points[-window:])
    weights = np.linspace(1, 2, window)
    weights /= weights.sum()
    smoothed = np.average(arr, axis=0, weights=weights)
    return tuple(smoothed.astype(int))

# =========================
# CARREGAR MODELO
# =========================
model = YOLO(model_path)

# Estruturas
trajectories_video = defaultdict(list)
trajectories_planta = defaultdict(list)

# =========================
# LOOP DE RASTREAMENTO
# =========================
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer = None
if SAVE_VIDEO:
    out_path = os.path.join(OUTPUT_DIR, "video_combinado_mask3_cortado.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, fps, (w + planta_img.shape[1], h))

for result in model.track(
    source=video_path,
    imgsz=imgsz,
    conf=conf,
    device=device,
    tracker=tracker_cfg,
    stream=True,
    persist=True,
    classes=[0]  # apenas pessoas
):
    frame = result.orig_img.copy()

    boxes = result.boxes
    if boxes is None or boxes.id is None:
        continue

    ids = boxes.id.int().cpu().tolist()
    xyxy = boxes.xyxy.cpu().numpy().astype(int)

    for i, track_id in enumerate(ids):
        x1, y1, x2, y2 = xyxy[i]
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        trajectories_video[track_id].append((cx, cy))
        smooth_cx, smooth_cy = smooth_point_weighted(trajectories_video[track_id])

        ponto_vid = np.array([[[smooth_cx, smooth_cy]]], dtype=np.float32)
        ponto_pla = cv2.perspectiveTransform(ponto_vid, H)[0][0]
        px_planta, py_planta = int(ponto_pla[0]), int(ponto_pla[1])
        trajectories_planta[track_id].append((px_planta, py_planta))

        color = color_by_id(track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
        draw_trajectory(frame, trajectories_video[track_id], color, LINE_THICKNESS)
        cv2.circle(frame, (cx, cy), 4, color, -1)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        draw_trajectory(planta_acumulada, trajectories_planta[track_id], color, LINE_THICKNESS)
        cv2.circle(planta_acumulada, (px_planta, py_planta), 4, color, -1)

    planta_resized = cv2.resize(planta_acumulada, (planta_acumulada.shape[1], h))
    combined = np.zeros((h, w + planta_resized.shape[1], 3), dtype=np.uint8)
    combined[:h, :w] = frame
    combined[:h, w:w+planta_resized.shape[1]] = planta_resized



    if SHOW_WINDOW:
        cv2.imshow("Video + Planta", combined)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if SAVE_VIDEO:
        writer.write(combined)

cap.release()
if writer:
    writer.release()
cv2.destroyAllWindows()

# =========================
# SALVAR PLANTA COM RASTROS
# =========================
planta_saida = os.path.join(OUTPUT_DIR, "planta_com_rastros_mask_cortado.png")
cv2.imwrite(planta_saida, planta_acumulada)
print(f"Planta com rastros salva em: {planta_saida}")
print("Processo concluído! O vídeo combinado foi salvo e a planta acumulada também.")






video 1/1 (frame 1/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 2029.2ms
video 1/1 (frame 2/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1941.7ms
video 1/1 (frame 3/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1820.3ms
video 1/1 (frame 4/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1841.1ms
video 1/1 (frame 5/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1946.4ms
video 1/1 (frame 6/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1925.8ms
video 1/1 (frame 7/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1905.0ms
video 1/1 (frame 8/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 2125.5ms
video 1/1 (frame 9/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 1932.6ms
video 1/1 (frame 10/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 2235.1ms
video 1/1 (frame 11/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 2375.9ms
video 1/1 (frame 12/3614) d:\YOLO\video cortado.mp4: 1280x1280 3 persons, 2184.1ms
video 1/1 (f

problema: alguns IDs perdem o rastreamento por oclusão, o que resulta na mudança de ID e cores.

solução:

Lógica de reassociação de IDs (ID stitching) integrada.

* Um ID canônico por pessoa (mesmo que o ByteTrack mude o ID bruto).
* Previsão por velocidade para reassociar com mais robustez.
* Janela temporal baseada no FPS do vídeo.
* Limiar adaptativo baseado no tamanho da caixa.
* Resolução de conflitos por frame (não atribui o mesmo ID canônico para duas detecções no mesmo frame).

In [1]:
import os
from collections import defaultdict
import numpy as np
import cv2
from ultralytics import YOLO

# ============================================================
# SISTEMA DE RASTREAMENTO + PROJEÇÃO EM PLANTA COM ID STITCHING
# ============================================================
# Este script:
# 1) Detecta e rastreia pessoas em 'video_path' com YOLOv8 + ByteTrack
# 2) Projeta o centro dos alvos para a planta (via homografia)
# 3) Mantém IDs estáveis mesmo quando o rastreador troca o ID (ID stitching)
# 4) Gera vídeo combinado (vídeo + planta) e a planta acumulada com trajetórias

# =========================
# CONFIGURAÇÕES
# =========================
video_path = "video cortado.MP4"
model_path = "yolov8l.pt"          # modelo de detecção (pode usar 'yolov8x.pt' p/ mais robustez)
tracker_cfg = "bytetrack.yaml"     # config do ByteTrack
imgsz = 920                       # resolução de inferência
conf = 0.40                        # confiança mínima
device = "cpu"                     # use "cuda" se tiver GPU

# Visual
LINE_THICKNESS = 2
BOX_THICKNESS = 2
SHOW_WINDOW = False
SAVE_VIDEO = True
OUTPUT_DIR = "runs/refino3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# PONTOS PARA HOMOGRAFIA
# =========================
pts_video = np.array([
    (16, 2886), (3223, 2630), (3820, 2253), (3223, 1376), (2360, 1360),
    (743, 1330), (1173, 900), (2010, 956), (2760, 990), (1313, 790), (1816, 790)
], dtype=np.float32)

pts_planta = np.array([
    (27, 770), (284, 771), (522, 772), (525, 541), (292, 540),
    (31, 537), (32, 305), (284, 298), (524, 302), (34, 30), (282, 31)
], dtype=np.float32)

H, _ = cv2.findHomography(pts_video, pts_planta)

# Carregar imagem da planta
planta_img = cv2.imread("mask_planta.png")
if planta_img is None:
    raise FileNotFoundError("Não foi possível carregar 'mask_planta.png'. Verifique o caminho.")
planta_acumulada = planta_img.copy()

# =========================
# FUNÇÕES AUXILIARES
# =========================
def color_by_id(track_id: int):
    np.random.seed(int(track_id) * 1117)
    c = np.random.randint(0, 255, 3).tolist()
    return tuple(map(int, c))

def draw_trajectory(frame, points, color, thickness=2):
    if len(points) > 1:
        pts = np.array(points, dtype=np.int32)
        cv2.polylines(frame, [pts], False, color, thickness)

def smooth_point_weighted(points, window=5):
    if len(points) < window:
        return points[-1]
    arr = np.array(points[-window:])
    weights = np.linspace(1, 2, window)
    weights /= weights.sum()
    smoothed = np.average(arr, axis=0, weights=weights)
    return tuple(smoothed.astype(int))

# =========================
# REASSOCIAÇÃO DE IDs (ID STITCHING)
# =========================
# Hiperparâmetros (ajuste conforme seu vídeo/FPS)
REID_ENABLED = True               # habilita a reassociação
MAX_MISSING_TIME_S = 1.0          # tempo (em segundos) para "esperar" o ID voltar
BASE_MAX_DIST = 80                # distância base (em pixels) para reassociar
USE_VELOCITY_PRED = True          # usa previsão de posição via velocidade

# Estruturas auxiliares
frame_idx = -1
id_alias = {}        # mapeia ID bruto (tracker) -> ID canônico estável
last_seen = {}       # última observação por ID canônico
# last_seen[cid] = {"pos": (cx, cy), "ts": frame_idx, "vel": (vx, vy), "box": (x1, y1, x2, y2)}

def _norm(v):
    return float(np.linalg.norm(np.array(v, dtype=np.float32)))

def _predict_pos(info, age_frames: int):
    """Prevê a posição com base na última posição + velocidade (se houver)."""
    pred = np.array(info["pos"], dtype=np.float32)
    if USE_VELOCITY_PRED and info.get("vel") is not None:
        pred = pred + age_frames * np.array(info["vel"], dtype=np.float32)
    return pred

def _gate_threshold(box, base=BASE_MAX_DIST):
    """Limiar adaptativo em função do tamanho da caixa (mais tolerância para caixas grandes)."""
    bw = max(1, box[2] - box[0])
    bh = max(1, box[3] - box[1])
    adaptive = 0.5 * (bw + bh) * 0.6  # ganho ~60% do tamanho médio
    return max(base, adaptive)

def reassign_id(
    raw_id: int,
    pos_xy: tuple,
    box_xyxy: tuple,
    frame_idx: int,
    last_seen: dict,
    id_alias: dict,
    max_missing_frames: int,
    claimed_canonical_ids: set,
):
    """
    Decide um ID canônico para o ID bruto do tracker:
      - Se já houver alias, reutiliza
      - Senão, busca um canônico recente cujo ponto previsto esteja próximo
      - Evita conflitos (não retorna um canônico já usado neste frame)
    """
    if raw_id in id_alias:
        return id_alias[raw_id]

    best_id, best_score = None, float("inf")
    gate = _gate_threshold(box_xyxy)

    for cand_id, info in last_seen.items():
        age = frame_idx - info["ts"]
        if age <= 0 or age > max_missing_frames:
            continue  # muito tempo sem ver esse canônico
        if cand_id in claimed_canonical_ids:
            continue  # já atribuído neste frame

        pred = _predict_pos(info, age)
        d = _norm(np.array(pos_xy) - pred)

        if d < gate and d < best_score:
            best_id, best_score = cand_id, d

    if best_id is not None:
        id_alias[raw_id] = best_id
        claimed_canonical_ids.add(best_id)
        return best_id

    # Não encontrou candidato: cria novo canônico = raw_id
    id_alias[raw_id] = raw_id
    claimed_canonical_ids.add(raw_id)
    return raw_id

def update_last_seen(canonical_id: int, pos_xy: tuple, box_xyxy: tuple, frame_idx: int):
    """Atualiza memória da última observação do ID canônico, estimando velocidade suavizada."""
    info = last_seen.get(canonical_id)
    if info is None:
        last_seen[canonical_id] = {
            "pos": tuple(map(float, pos_xy)),
            "ts": frame_idx,
            "vel": None,
            "box": tuple(map(int, box_xyxy)),
        }
        return

    prev_pos = np.array(info["pos"], dtype=np.float32)
    prev_ts = info["ts"]
    dt = max(1, frame_idx - prev_ts)  # frames
    cur_pos = np.array(pos_xy, dtype=np.float32)

    inst_vel = (cur_pos - prev_pos) / dt
    prev_vel = np.array(info["vel"], dtype=np.float32) if info["vel"] is not None else np.zeros(2, dtype=np.float32)
    # EMA para suavizar velocidade
    alpha = 0.7
    new_vel = alpha * prev_vel + (1.0 - alpha) * inst_vel

    last_seen[canonical_id] = {
        "pos": tuple(map(float, pos_xy)),
        "ts": frame_idx,
        "vel": tuple(map(float, new_vel)),
        "box": tuple(map(int, box_xyxy)),
    }

def prune_last_seen(frame_idx: int, max_missing_frames: int):
    """Limpa entradas muito antigas (apenas para não crescer indefinidamente)."""
    to_del = []
    horizon = max_missing_frames * 3
    for cid, info in list(last_seen.items()):
        if frame_idx - info["ts"] > horizon:
            to_del.append(cid)
    for cid in to_del:
        last_seen.pop(cid, None)

# =========================
# CARREGAR MODELO
# =========================
model = YOLO(model_path)

# Estruturas de trajetória
trajectories_video = defaultdict(list)
trajectories_planta = defaultdict(list)

# =========================
# LOOP DE RASTREAMENTO
# =========================
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Não foi possível abrir o vídeo '{video_path}'.")

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Ajusta janela temporal pela taxa de quadros
if fps is None or fps <= 0:
    fps = 30.0  # fallback
max_missing_frames = int(MAX_MISSING_TIME_S * fps)

writer = None
if SAVE_VIDEO:
    out_path = os.path.join(OUTPUT_DIR, "video_combinado_mask3_cortado.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # largura combinada = vídeo (w) + planta (mesma largura da imagem de planta original redimensionada para h)
    writer = cv2.VideoWriter(out_path, fourcc, fps, (w + planta_img.shape[1], h))

# Importante: usar stream=True e persist=True para manter estado do tracker entre frames
for result in model.track(
    source=video_path,
    imgsz=imgsz,
    conf=conf,
    device=device,
    tracker=tracker_cfg,
    stream=True,
    persist=True,
    classes=[0]  # apenas pessoas
):
    # Avança o índice de frame SEMPRE (mesmo se não houver detecção)
    frame_idx += 1

    frame = result.orig_img.copy()
    boxes = result.boxes

    # Conjunto para evitar dar o mesmo canônico a 2 detecções no mesmo frame
    claimed_canonical_ids = set()

    if boxes is None or boxes.id is None:
        # Nenhuma detecção: ainda assim podemos mostrar/gravar o frame atual (opcional)
        # Para manter igual ao seu fluxo anterior (que 'continue'), descomente a linha abaixo.
        # continue
        # -> Aqui optamos por escrever/mostrar o frame para consistência temporal do vídeo:
        planta_resized = cv2.resize(planta_acumulada, (planta_acumulada.shape[1], h))
        combined = np.zeros((h, w + planta_resized.shape[1], 3), dtype=np.uint8)
        combined[:h, :w] = frame
        combined[:h, w:w+planta_resized.shape[1]] = planta_resized

        if SHOW_WINDOW:
            cv2.imshow("Video + Planta", combined)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        if SAVE_VIDEO and writer is not None:
            writer.write(combined)

        prune_last_seen(frame_idx, max_missing_frames)
        continue

    ids = boxes.id.int().cpu().tolist()
    xyxy = boxes.xyxy.cpu().numpy().astype(int)

    for i, raw_id in enumerate(ids):
        x1, y1, x2, y2 = xyxy[i]
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        # ===== REASSOCIAÇÃO DE ID =====
        if REID_ENABLED:
            canonical_id = reassign_id(
                raw_id=raw_id,
                pos_xy=(cx, cy),
                box_xyxy=(x1, y1, x2, y2),
                frame_idx=frame_idx,
                last_seen=last_seen,
                id_alias=id_alias,
                max_missing_frames=max_missing_frames,
                claimed_canonical_ids=claimed_canonical_ids,
            )
        else:
            canonical_id = raw_id

        # Atualiza memória do "visto por último" com o ID canônico
        update_last_seen(
            canonical_id=canonical_id,
            pos_xy=(cx, cy),
            box_xyxy=(x1, y1, x2, y2),
            frame_idx=frame_idx,
        )

        # ===== A PARTIR DAQUI, USE SEMPRE O ID CANÔNICO =====
        trajectories_video[canonical_id].append((cx, cy))
        smooth_cx, smooth_cy = smooth_point_weighted(trajectories_video[canonical_id])

        # Projeta ponto suavizado para a planta
        ponto_vid = np.array([[[smooth_cx, smooth_cy]]], dtype=np.float32)
        ponto_pla = cv2.perspectiveTransform(ponto_vid, H)[0][0]
        px_planta, py_planta = int(ponto_pla[0]), int(ponto_pla[1])
        trajectories_planta[canonical_id].append((px_planta, py_planta))

        # Cor por ID canônico
        color = color_by_id(canonical_id)

        # Desenhos no vídeo
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICKNESS)
        draw_trajectory(frame, trajectories_video[canonical_id], color, LINE_THICKNESS)
        cv2.circle(frame, (cx, cy), 4, color, -1)
        cv2.putText(frame, f"ID {canonical_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Desenhos na planta acumulada
        draw_trajectory(planta_acumulada, trajectories_planta[canonical_id], color, LINE_THICKNESS)
        cv2.circle(planta_acumulada, (px_planta, py_planta), 4, color, -1)

    # Monta lado a lado: vídeo + planta
    planta_resized = cv2.resize(planta_acumulada, (planta_acumulada.shape[1], h))
    combined = np.zeros((h, w + planta_resized.shape[1], 3), dtype=np.uint8)
    combined[:h, :w] = frame
    combined[:h, w:w+planta_resized.shape[1]] = planta_resized

    if SHOW_WINDOW:
        cv2.imshow("Video + Planta", combined)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if SAVE_VIDEO and writer is not None:
        writer.write(combined)

    # Limpeza opcional ao fim de cada frame
    prune_last_seen(frame_idx, max_missing_frames)

cap.release()
if writer:
    writer.release()
cv2.destroyAllWindows()

# =========================
# SALVAR PLANTA COM RASTROS
# =========================
planta_saida = os.path.join(OUTPUT_DIR, "planta_com_rastros_mask_cortado.png")
cv2.imwrite(planta_saida, planta_acumulada)
print(f"Planta com rastros salva em: {planta_saida}")



WARNING imgsz=[920] must be multiple of max stride 32, updating to [928]
video 1/1 (frame 1/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1057.5ms
video 1/1 (frame 2/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1109.0ms
video 1/1 (frame 3/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1113.9ms
video 1/1 (frame 4/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1063.4ms
video 1/1 (frame 5/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1112.5ms
video 1/1 (frame 6/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1042.2ms
video 1/1 (frame 7/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1067.2ms
video 1/1 (frame 8/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1119.3ms
video 1/1 (frame 9/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1079.4ms
video 1/1 (frame 10/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1156.8ms
video 1/1 (frame 11/3614) d:\YOLO\video cortado.MP4: 928x928 2 persons, 1211.1ms
video 1/1 (frame 12/3614) d:\YOLO\video cort

# Usando a placa de video

GPU: NVIDIA RTX A1000 (6 GB)

Driver: 553.46

CUDA versão: 12.4

ativar ambiente virtual: .\yolovenv\Scripts\activate

Remover Torch CPU-only e limpar cache: pip uninstall -y torch torchvision torchaudio
pip cache purge

Instalar PyTorch com suporte CUDA: pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

Instalar Ultralytics e dependências: pip install --upgrade ultralytics opencv-python numpy

Testar se a GPU está ativa: python -c "import torch; print('CUDA disponível:', torch.cuda.is_available()); print('Versão CUDA:', torch.version.cuda); print('GPUs:', torch.cuda.device_count())"

